In [36]:
#Importing required modules
import math as math
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
from galpy.potential import MWPotential2014
from galpy.actionAngle import actionAngleAdiabatic
from galpy.df import quasiisothermaldf
from galpy.actionAngle import actionAngleStaeckel
from scipy.interpolate import RectBivariateSpline as interpolation

#Setting up the action angle and qdf with MWPotential2014
aAS= actionAngleStaeckel(pot=MWPotential2014,delta=0.45,c=True)
qdfS= quasiisothermaldf(1./3.,0.2,0.1,1.,1.,pot=MWPotential2014,aA=aAS,cutcounter=True)

#Defining a function to extract a sub-matrix we can interpolate
def getSubMatrix(parentMatrix, spacing, offset):
    curr_row = offset
    curr_column = offset
    shape = parentMatrix.shape

    a = np.zeros((math.ceil((shape[0] - offset) / spacing), math.ceil((shape[1] - offset) / spacing)))

    new_row = 0
    new_column = 0

    while (curr_column < shape[1]):
        while (curr_row < shape[0]):
            #print("new row " , new_row , " new column " , new_column , " curr_row " , curr_row , " curr_column " , curr_column)
            a[new_row][new_column] = parentMatrix[curr_row][curr_column]
            curr_row += spacing
            new_row += 1

        curr_column += spacing
        new_column += 1
        new_row = 0
        curr_row = offset
        #print("curr_column ", curr_column, " new_column ", new_column, " new_row ", new_row, " curr_row ", curr_row)

    return a

In [38]:
#Setting up R and z values within a 1 kpc sphere of the Sun
R =  np.arange(7., 9., 0.2)*u.kpc
z =  np.arange(-1., 1., 0.2)*u.kpc

#Using a loop to compute vR, vT, and vz arrays from sampleV
new_vR = np.empty((10, 10))
new_vT = np.empty((10, 10))
new_vz = np.empty((10, 10))
for i in range(len(R)):
    for j in range(len(z)):
        s = qdfS.sampleV(R[i], z[j], n=1)
        new_vR[i][j] = s[0, 0]
        new_vT[i][j] = s[0, 1]
        new_vz[i][j] = s[0, 2]

c:\users\ayush\anaconda3\lib\site-packages\galpy\df_src\quasiisothermaldf.py:266: RuntimeWarning: divide by zero encountered in log
  -kappa*jr*numpy.exp(-2.*lnsr)

Optimization terminated successfully.
         Current function value: -2.437856
         Iterations: 2
         Function evaluations: 39
Optimization terminated successfully.
         Current function value: -3.591801
         Iterations: 2
         Function evaluations: 40
Optimization terminated successfully.
         Current function value: -4.604754
         Iterations: 2
         Function evaluations: 42
Optimization terminated successfully.
         Current function value: -5.440985
         Iterations: 2
         Function evaluations: 39
Optimization terminated successfully.
         Current function value: -6.038605
         Iterations: 1
         Function evaluations: 14
Optimization terminated successfully.
         Current function value: -6.273111
         Iterations: 2
         Function evaluations: 30
Optimiz

Optimization terminated successfully.
         Current function value: -3.786049
         Iterations: 2
         Function evaluations: 39
Optimization terminated successfully.
         Current function value: -4.713105
         Iterations: 2
         Function evaluations: 40
Optimization terminated successfully.
         Current function value: -5.478769
         Iterations: 2
         Function evaluations: 49
Optimization terminated successfully.
         Current function value: -6.025436
         Iterations: 2
         Function evaluations: 68
Optimization terminated successfully.
         Current function value: -6.239659
         Iterations: 2
         Function evaluations: 39
Optimization terminated successfully.
         Current function value: -6.025436
         Iterations: 2
         Function evaluations: 68
Optimization terminated successfully.
         Current function value: -5.478769
         Iterations: 2
         Function evaluations: 51
Optimization terminated successful

In [63]:
#Setting up new R and z values coresponding to sub-matrices
sub_R =  np.arange(7., 9., 0.6)*u.kpc
sub_z =  np.arange(-1., 1., 0.6)*u.kpc

#Creating sub-matrices
sub_vR = getSubMatrix(new_vR, 3, 0)
sub_vT = getSubMatrix(new_vT, 3, 0)
sub_vz = getSubMatrix(new_vz, 3, 0)

#Get interpolation objects with sub-matrices
ip_vR = interpolation(sub_R, sub_z, sub_vR)
ip_vT = interpolation(sub_R, sub_z, sub_vT)
ip_vz = interpolation(sub_R, sub_z, sub_vz)

#Interpolating at points from interpolation objects
interp_vR = np.empty((10, 10))
interp_vT = np.empty((10, 10))
interp_vz = np.empty((10, 10))
for i in range(len(R)):
    for j in range(len(z)):
        interp_vR[i][j] = ip_vR.ev(R[i], z[j])
        interp_vT[i][j] = ip_vT.ev(R[i], z[j])
        interp_vz[i][j] = ip_vz.ev(R[i], z[j])

#Find the difference between interpolated and original data
delta_vR = new_vR - interp_vR
delta_vT = new_vT - interp_vT
delta_vz = new_vz - interp_vz

#Get error in each velocity component
error_vR = np.abs(delta_vR / new_vR)
error_vT = np.abs(delta_vT / new_vT)
error_vz = np.abs(delta_vz / new_vz)

#Print the mean fractional error in each velocity component
#vR and vz are expected to be high due to distribution around 0 but vT should accurately represent the fractional error
print(np.mean(error_vR), np.mean(error_vT), np.mean(error_vz))

3.0724103066620807 0.15771579475764944 2.118219494031699
